In [1]:
from __future__ import division
import numpy as np
from matplotlib import pyplot as plt

# Generate the data according to the specification in the homework description

N = 500
x = np.random.rand(N).reshape(500,1)

pi0 = np.array([0.7, 0.3])
w0 = np.array([-2, 1]).reshape(1,2)
b0 = np.array([0.5, -0.5]).reshape(1,2)
sigma0 = np.array([.4, .3])

y = np.zeros_like(x)
for i in range(N):
    k = 0 if np.random.rand() < pi0[0] else 1
    y[i] = w0[0,k]*x[i] + b0[0,k] + np.random.randn()*sigma0[k]


# TODO: Implement the EM algorithm for Mixed Linear Regression based on observed
# x and y values.


# Here's the data plotted
plt.scatter(x, y, c='r', marker='x')
plt.show()

    

In [2]:
#rebuilt the data
N=500
Intercept=np.ones((y.size,1))
x1=np.concatenate((x,Intercept),axis=1)
#w1=np.array([1,-1]).reshape(1,2)
#b1=np.array([0,0]).reshape(1,2)
wt1=np.array([1,0]).reshape(2,1)
wt2=np.array([-1,0]).reshape(2,1)
pi1=np.array([0.5,0.5])
#wt=np.concatenate((w1,b1))
sigma1=np.array([np.std(y),np.std(y)])
from scipy.stats import norm
R1=np.zeros(N)
R2=np.zeros(N)
for i in range(N):
    R1[i]=pi1[0]*(norm.pdf(y[i],np.dot(x1[i,:].reshape(1,2),wt1),sigma1[0]))/(pi1[0]*(norm.pdf(y[i],np.dot(x1[i,:].reshape(1,2),wt1),sigma1[0]))+pi1[1]*(norm.pdf(y[i],np.dot(x1[i,:].reshape(1,2),wt2),sigma1[1])))
    R2[i]=1-R1[i]
#maxlikelihood
def lg(pi1,w1,w2,x1,sigma):
    a=0
    for i in range(N):
        a=a+np.log(pi1[0]*(norm.pdf(y[i],np.dot(x1[i,:].reshape(1,2),w1),sigma[0]))+pi1[1]*(norm.pdf(y[i],np.dot(x1[i,:].reshape(1,2),w2),sigma[1])))
    return a
#update
def pi(R1,R2,N):
    b=np.zeros(2)
    b[0]=np.sum(R1)/N
    b[1]=np.sum(R2)/N
    return b

def w(R1,R2,x1,y):
    R11=np.diag(R1)
    R22=np.diag(R2)   
    w11=np.dot(np.dot(np.dot(np.linalg.inv(np.dot(np.dot(x1.T,R11),x1)),x1.T),R11),y)
    w12=np.dot(np.dot(np.dot(np.linalg.inv(np.dot(np.dot(x1.T,R22),x1)),x1.T),R22),y)
    return w11,w12

def s(R1,R2,y,w1,w2,x1):
    s=np.zeros(2)
    R11=np.diag(R1)
    R22=np.diag(R2) 
    s[0]=np.sqrt((x1.dot(w1)-y).T.dot(R11).dot(x1.dot(w1)-y)/np.sum(R1))
    s[1]=np.sqrt((x1.dot(w2)-y).T.dot(R22).dot(x1.dot(w2)-y)/np.sum(R2))
    return s
j=0
lg0=lg(pi1,wt1,wt2,x1,sigma1)
pi1=pi(R1,R2,N)
wt1,wt2=w(R1,R2,x1,y)
sigma1=s(R1,R2,y,wt1,wt2,x1)
lg1=np.append(lg0,lg(pi1,wt1,wt2,x1,sigma1))
while np.abs(lg1[j+1]-lg1[j])>10**(-4):
    for i in range(N):
        R1[i]=pi1[0]*(norm.pdf(y[i],np.dot(x1[i,:].reshape(1,2),wt1),sigma1[0]))/(pi1[0]*(norm.pdf(y[i],np.dot(x1[i,:].reshape(1,2),wt1),sigma1[0]))+pi1[1]*(norm.pdf(y[i],np.dot(x1[i,:].reshape(1,2),wt2),sigma1[1])))
        R2[i]=1-R1[i]
    pi1=pi(R1,R2,N)
    wt1,wt2=w(R1,R2,x1,y)
    sigma1=s(R1,R2,y,wt1,wt2,x1)
    lg1=np.append(lg1,lg(pi1,wt1,wt2,x1,sigma1))
    j=j+1

In [4]:
print pi1,wt1,wt2,sigma1

[ 0.30606732  0.69393268] [[ 1.00135633]
 [-0.50474549]] [[-1.95295001]
 [ 0.50580164]] [ 0.30613885  0.39743446]


In [5]:
M=np.arange(lg1.size)
print M
plt.plot(M,lg1)
plt.show()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [ ]:
plt.scatter(x, y, c='r', marker='x')
plt.scatter(x,np.dot(x1,wt1))
plt.scatter(x,np.dot(x1,wt2))
plt.show()

[[-533.68714225]] [[-389.35813989]] [[-372.0022108]] [[-364.93585225]]


[[-533.68714225]] [[-389.35813989]] [[-372.0022108]] [[-360.48236703]]


[[-533.68714225]] [[-389.35813989]] [[-372.0022108]] [[-357.94892897]]
